In [1]:
import os
os.chdir('..')

In [2]:
import scipy.stats as stats 
import numpy as np
import pandas as pd
import torch
import random
import sys
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import time
from tqdm import tqdm
from pathlib import Path


from hac.utils.key_points import W_LIST_POSE, W2I_POSE, \
                                 W_LIST_LEFT_HAND, W2I_LEFT_HAND, \
                                 W_LIST_RIGHT_HAND, W2I_RIGHT_HAND, \
                                 X_COLS, Y_COLS, \
                                 ALL_XY_COLS, HAND_XY_COLS, \
                                 LEFT_HAND_X_COLS, LEFT_HAND_Y_COLS, \
                                 RIGHT_HAND_X_COLS, RIGHT_HAND_Y_COLS
from hac.utils.normalizer import normalize_data, normalize_hand_data

In [3]:
os.getcwd()

'C:\\Users\\JAQQ\\YOLO\\hac'

In [4]:
torch.random.manual_seed(5566)
np.random.seed(5566)
random.seed(5566)

In [5]:
data_path = "data\\actions"
model_target = "gestures"

if model_target == "actions":
    mode = "pose_hand"
    model_name = "roblox_lift_game"
    actions = ["walk", "jump", "hands_on_hips", "point_left", "point_right", "arms_lift", "punch", "trample", "lateral_raise", "stand"]
    target_columns = ALL_XY_COLS
    target_columns_x = target_columns.copy()
    target_columns += ["image_name", "label"]
elif model_target == "gestures":
    mode = "hand"
    model_name = "mouse"
    actions = ["r_five", "r_zero", "l_five", "l_zero", "two_index_fingers_up", "two_index_fingers_down", "33", "55", "sit", "gesture_none"]
    target_columns = [key + "_x" for key in W_LIST_RIGHT_HAND] + [key + "_y" for key in W_LIST_RIGHT_HAND]
    target_columns += [key + "_x" for key in W_LIST_LEFT_HAND] + [key + "_y" for key in W_LIST_LEFT_HAND]
    target_columns_x = target_columns.copy()
    target_columns += ["image_name", "label"]
else:
    RunTimeError("???")

dfs = []
for idx, action in enumerate(actions):
    file_path = os.path.join(data_path, action, "data.csv")
    print(file_path)
    df = pd.read_csv(file_path)

    df.label = idx
    dfs.append(df)

data\actions\r_five\data.csv
data\actions\r_zero\data.csv
data\actions\l_five\data.csv
data\actions\l_zero\data.csv
data\actions\two_index_fingers_up\data.csv
data\actions\two_index_fingers_down\data.csv
data\actions\33\data.csv
data\actions\55\data.csv
data\actions\sit\data.csv
data\actions\gesture_none\data.csv


In [6]:
df_train = pd.concat(dfs)
df_train = df_train.reset_index(drop=True)
df_train

,n_x,n_y,n_v,lei_x,lei_y,lei_v,le_x,le_y,le_v,leo_x,...,r_p_p_y,r_p_p_v,r_p_d_x,r_p_d_y,r_p_d_v,r_p_t_x,r_p_t_y,r_p_t_v,image_name,label
0,0.418780,0.480716,0.999984,0.442415,0.543997,0.999991,0.458284,0.540193,0.999973,0.473180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1626055861271.png,0
1,0.415956,0.480416,0.999984,0.439260,0.544007,0.999990,0.454108,0.540206,0.999972,0.470344,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1626055861583.png,0
2,0.413536,0.480286,0.999984,0.437101,0.544014,0.999990,0.451680,0.540220,0.999971,0.468653,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1626055861879.png,0
3,0.414042,0.479995,0.999985,0.437432,0.543289,0.999990,0.452207,0.539171,0.999971,0.468828,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1626055862177.png,0
4,0.414334,0.479671,0.999984,0.437617,0.542777,0.999990,0.452594,0.538495,0.999971,0.468921,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1626055862478.png,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6162,0.490220,0.803613,0.999711,0.500282,0.826710,0.999628,0.507361,0.825209,0.999545,0.513091,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1625134864956.png,9
6163,0.492461,0.804014,0.999715,0.501522,0.826775,0.999638,0.508283,0.825258,0.999551,0.513766,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1625134865282.png,9
6164,0.492778,0.806098,0.999741,0.501741,0.828155,0.999669,0.508624,0.826842,0.999591,0.514198,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1625134865657.png,9
6165,0.493009,0.803616,0.999747,0.502379,0.825692,0.999683,0.509299,0.824126,0.999573,0.514788,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1625134865971.png,9


In [7]:
if model_target == "actions":
    df_train = normalize_data(df_train)
    display(df_train)
if model_target == "gestures":
    df_train = normalize_hand_data(df_train)
    display(df_train)

,l_w_x,l_t_c_x,l_t_m_x,l_t_i_x,l_t_t_x,l_i_m_x,l_i_p_x,l_i_d_x,l_i_t_x,l_m_m_x,...,r_r_m_y,r_r_p_y,r_r_d_y,r_r_t_y,r_p_m_y,r_p_p_y,r_p_d_y,r_p_t_y,image_name,label
0,0.0,-0.045952,-0.077469,-0.097663,-0.128939,-0.056310,-0.068727,-0.075495,-0.078445,-0.025674,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1626055861271.png,0
1,0.0,-0.043304,-0.077839,-0.101611,-0.129572,-0.050289,-0.066453,-0.072824,-0.075547,-0.020425,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1626055861583.png,0
2,0.0,-0.042498,-0.077526,-0.102028,-0.127668,-0.048393,-0.064645,-0.070264,-0.071851,-0.019090,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1626055861879.png,0
3,0.0,-0.042338,-0.074218,-0.098538,-0.124541,-0.040349,-0.054675,-0.058814,-0.059319,-0.012223,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1626055862177.png,0
4,0.0,-0.038826,-0.067925,-0.088480,-0.112362,-0.035974,-0.047616,-0.050558,-0.049466,-0.007391,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1626055862478.png,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6162,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1625134864956.png,9
6163,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1625134865282.png,9
6164,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1625134865657.png,9
6165,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1625134865971.png,9


In [8]:
df_train_x = df_train[target_columns_x]
df_train_y = df_train["label"]
df_train_image = df_train["image_name"]

In [9]:
class Graph:
        
    def __init__(self, 
                 k_hop, mode):
        
        """
        inputs:
            mode: "hand", "pose", "pose_hand"
        """
        
        self.k_hop = k_hop
        self.mode = mode
        
        self.create_vertex()
        self.create_edge()
        self.create_adjacent_matrix()
        self.create_D()
        self.create_k_hop_matrix()
        
    def create_vertex(self):
        self.pose_num_vertices = 33
        self.hand_num_vertices = 21
        if self.mode == "hand":
            self.num_vertices = self.hand_num_vertices * 2
        elif self.mode == "pose":
            self.num_vertices = self.pose_num_vertices
        elif self.mode == "pose_hand":
            self.num_vertices = self.pose_num_vertices + \
                                self.hand_num_vertices * 2
            
            
        
    def create_edge(self):
        self_edges = [(v, v) for v in range(self.num_vertices)]
        neighbor_edges = [(0, 1), (0, 4), (1, 2), (2, 3),
                         (3, 7), (4, 5), (5, 6), (6, 8), 
                         (9, 10), (11, 12), (11, 13), (11, 23), (12, 14),
                         (12, 24), (13, 15), (14, 16), (15, 17), (15, 19),
                         (15, 21), (16, 18), (16, 20), (16, 22), (17, 19),
                         (18, 20), (23, 24), (23, 25), (24, 26), (25, 27),
                         (26, 28), (27, 29), (27, 31), (28, 30), (28, 32), 
                         (29, 31), (30, 32), 
                        ]
        
        hand_edges = [
            (0, 1), (0, 5), (0, 9), (0, 13), (0, 17),
            (1, 2), (2, 3), (3, 4), (5, 6), (5, 9), 
            (6, 7), (7, 8), (9, 10), (9, 13), (10, 11),
            (11, 12), (13, 14), (13, 17), (14, 15), (15, 16),
            (17, 18), (18, 19), (19, 20)
        ]
        
        if self.mode == "hand":
            left_hand_edges = hand_edges.copy()
            right_hand_edges = [(i+self.hand_num_vertices, j+self.hand_num_vertices) \
                                 for i, j in hand_edges]
            self.edges = self_edges + left_hand_edges + right_hand_edges
        elif self.mode == "pose":
            self.edges = self_edges + neighbor_edges
        elif self.mode == "pose_hand":
            left_hand_edges = [(i+self.pose_num_vertices, j+self.pose_num_vertices) \
                                 for i, j in hand_edges]
            right_hand_edges = [(i+self.hand_num_vertices, j+self.hand_num_vertices) \
                                 for i, j in left_hand_edges]
            self.edges = self_edges + neighbor_edges + left_hand_edges + right_hand_edges
        
    def create_adjacent_matrix(self):
        self.A = np.zeros((self.num_vertices, self.num_vertices))
        for i, j in self.edges:
            self.A[i, j] = 1
            self.A[j, i] = 1
        
    def create_D(self):
        sum_row = self.A.sum(axis=1)
        self.D = np.diag(sum_row)
        
    def get_normalized_A(self):
        return np.linalg.inv(self.D) @ self.A
    
    def create_k_hop_matrix(self):
        
        As = []
        
        for hop in range(1, self.k_hop + 1):
            As.append(np.power(self.A, hop))
        
        self.A_k = np.stack(As, axis=0)
        

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(self, in_channels, out_channels, k_hop):
        super().__init__()
        
        self.conv = nn.Conv2d(in_channels,
                              out_channels * k_hop,
                              kernel_size=(1, 1),
                              padding=(0, 0),
                              stride=(1, 1))
        
        if in_channels == out_channels:
            self.residual = lambda x: x

        else:
            self.residual = nn.Sequential(
                nn.Conv2d(
                    in_channels,
                    out_channels,
                    kernel_size=1,
                    stride=(1, 1)),
                nn.BatchNorm2d(out_channels),
            )
        
    def forward(self, x, A_k):
        
        res = self.residual(x)
        
        B, F, J, T = x.size()
        
        K = A_k.size()[0]
        
        x = self.conv(x)
        B, KC, J, T = x.size()
        
        x = x.view(B, K, KC // K, J, T)
        output = torch.einsum('bkcjt,kji->bcit', (x, A_k)) + res
        
        return output
    
class HACModel(nn.Module):
    def __init__(self, in_channels, num_class, k_hop, mode):
        super().__init__()
        
        self.graph = Graph(k_hop=k_hop, mode=mode)
        A_k = torch.tensor(self.graph.A_k,
                           dtype=torch.float32,
                           requires_grad=False)
        self.register_buffer('A_k', A_k)
        
        self.data_bn = nn.BatchNorm2d(in_channels)
        self.gcns = nn.ModuleList((
            GCN(in_channels, 16, k_hop),
            GCN(16, 32, k_hop),
            GCN(32, 64, k_hop),           
        ))
        
        self.edge_importances = nn.ParameterList([
            nn.Parameter(torch.ones(self.A_k.size()))
            for i in self.gcns
        ])
        
        self.fcn = nn.Conv2d(64, num_class, kernel_size=1)
        self.drop_layer = nn.Dropout(p=0.5)
        
    def forward(self, x):
        x = self.data_bn(x)
        for gcn, importance in zip(self.gcns, self.edge_importances):
            x = gcn(x, self.A_k * importance)
        x = F.avg_pool2d(x, x.size()[2:])
        x = self.drop_layer(x)
        output = self.fcn(x).squeeze()
        
        return output

In [11]:
in_channels = 2
num_class = len(actions)
model = HACModel(in_channels, num_class, k_hop=3, mode=mode).to('cuda')
if mode == "hand":
    j = 21 * 2 
elif mode == "pose":
    j = 33
elif mode == "pose_hand":
    j = 33 + 21 * 2 
x = torch.rand((32, 2, j, 1)).to('cuda')
s = time.time()
output = model(x)
print(time.time() - s)
print(output.size())
print(torch.argmax(output,dim=1,keepdim=True))

1.3100800514221191
torch.Size([32, 10])
tensor([[9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [3],
        [1],
        [9],
        [1],
        [3],
        [9],
        [4],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [9],
        [5],
        [9]], device='cuda:0')


In [12]:
'''
f1 = lambda key: key + "_x"
f2 = lambda key: key + "_y"

pose_cols = [f(key) for key in W_LIST_POSE for f in (f1, f2)]
pose_cols
'''

'\nf1 = lambda key: key + "_x"\nf2 = lambda key: key + "_y"\n\npose_cols = [f(key) for key in W_LIST_POSE for f in (f1, f2)]\npose_cols\n'

In [13]:
import torch.utils.data as data

In [14]:
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
        assert self.X.shape[0] == self.y.shape[0]

    def __len__(self):
        
        return self.y.shape[0]
    
    def __getitem__(self, idx):

        return self.X[idx,:], self.y[idx]

In [15]:
device = 'cuda'

In [16]:
def train(model, train_loader, val_loader, optimizer, 
          loss_fn, device, num_epochs, model_path):

    model.train()
    
    accs = []
    losses = []
    val_accs = []
    val_losses = []
    best_val_acc = 0.0

    for epoch in tqdm(range(0, num_epochs)):
        epoch_loss = 0.0
        epoch_acc = 0.0
        count = 0
        for x, y in train_loader:
            x = x.view(-1, 2, x.shape[1]//2, 1).to(device).float()
            y = y.to(device)

            optimizer.zero_grad()
            pred = model(x).reshape((y.shape[0], -1))
            loss = loss_fn(pred, y)      
            
            epoch_loss += loss.item()
            loss.backward()

            epoch_acc += (pred.argmax(axis=1, keepdim=True).squeeze() == y).sum().item()
            optimizer.step()
            count += x.size()[0]

        epoch_loss /= len(train_loader)
        epoch_acc /= count
        losses.append(epoch_loss)
        accs.append(epoch_acc)
        print("epoch loss:", epoch_loss)
        print("acc:", epoch_acc)
        
        if val_loader:
            val_loss, val_acc = evaluate(model, val_loader, loss_fn, device)
            val_losses.append(val_loss)
            val_accs.append(val_acc)
            
        if val_loader and val_loss > best_val_acc:
            model_name = "best_model.pth"
            Path(model_path).mkdir(parents=True, exist_ok=True)
            torch.save(model.state_dict(), 
                       os.path.join(model_path, model_name))
            
    return losses, accs, val_losses, val_accs

def evaluate(model, val_loader, loss_fn, device):

    model.eval()
    
    val_loss = 0.0
    val_acc = 0.0
    count = 0
    for x, y in val_loader:
        x = x.view(-1, 2, x.shape[1]//2, 1).to(device).float()
        y = y.to(device)

        pred = model(x).reshape((y.shape[0], -1))
        loss = loss_fn(pred, y)
        val_loss += loss.item()

        val_acc += (pred.argmax(axis=1, keepdim=True).squeeze() == y).sum().item()
        count += x.size()[0]
        
    val_loss /= len(val_loader)
    val_acc /= count
    print("val loss:", val_loss)
    print("val acc:", val_acc)
        
    return val_loss, val_acc

In [ ]:
from sklearn.model_selection import StratifiedKFold
from collections import Counter
hop = 2
batch_size = 8
num_epochs = 100
num_class = len(actions)
skf = StratifiedKFold(n_splits=5)
model_path = "hac\\model\\gcn\\" + model_target

k_fold_losses = []
k_fold_accs = []
k_fold_val_losses = []
k_fold_val_accs = []

for train_index, test_index in skf.split(df_train[target_columns_x].values, df_train.label.values):
    X = df_train[target_columns_x].values[train_index]
    y = df_train["label"].values[train_index]
    val_X = df_train[target_columns_x].values[test_index]
    val_y = df_train["label"].values[test_index]
    
    dataset = Dataset(X, y)
    val_dataset = Dataset(val_X, val_y)
    
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    model = HACModel(in_channels, num_class, hop, mode).to('cuda')
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    losses, accs, val_losses, val_accs = train(model, train_loader, val_loader, optimizer, 
                                               loss_fn, device, num_epochs, model_path)
    k_fold_losses.append(losses)
    k_fold_accs.append(accs)
    
    k_fold_val_losses.append(val_losses)
    k_fold_val_accs.append(val_accs)
    break

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

epoch loss: 2.2898876259837
acc: 0.47314007703223193


  1%|▊                                                                                 | 1/100 [00:03<06:16,  3.80s/it]

val loss: 1.1155341002444226
val acc: 0.6969205834683955
epoch loss: 0.5435335636742691
acc: 0.8327589702006892


  2%|█▋                                                                                | 2/100 [00:07<05:58,  3.66s/it]

...

val loss: 1.9258420470041775
val acc: 0.8306320907617504
epoch loss: 0.3325933547995417
acc: 0.9176971416987634


  4%|███▎                                                                              | 4/100 [00:14<05:44,  3.58s/it]

val loss: 2.2274057975321804
val acc: 0.8063209076175041


  4%|███▎                                                                              | 4/100 [00:14<05:51,  3.67s/it]


KeyboardInterrupt: 

In [ ]:
for idx in range(len(k_fold_val_accs)):
    plt.figure()
    plt.plot(k_fold_accs[idx], label="train")
    plt.plot(k_fold_val_accs[idx], label="test")
    plt.legend()
    plt.show()

In [ ]:
'''
losses_k_hops = []
accs_k_hops = []

for hop in range(1, 6):
    
    model = HACModel(in_channels, num_class, hop).to('cuda')
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    losses, accs, _, _ = train(model, train_loader, None, optimizer, loss_fn, device, num_epochs)
    
    losses_k_hops.append(losses)
    accs_k_hops.append(accs)
'''

In [ ]:
'''
plt.figure()
for idx, accs in enumerate(accs_k_hops):
    plt.plot(accs, label=str(idx + 1))
    print(np.array(accs).mean())
plt.ylim(0.9, 1.0)
plt.legend()
plt.show()
'''